## Create a Program:

The program should simulate 1000 individual battle rounds in the board game Risk, specifically with 3 attackers vs. 2 defenders.

### Battle round rules: 
Each battle round involves one roll of dice for both attacker and defender. 
**The highest dice roll from each side is compared:**
If the attacker’s roll is equal to or lower than the defender’s, *the attacker loses one troop*.
If the attacker’s roll is higher, *the defender loses one troop*.
**Compare Second-Highest Dice Rolls**: The same rule. 

### Plot Results: 
The program should produce a plot summarizing the results of the battle rounds. The specifics of what to plot are left to your discretion.

Extra Challenge: Create a more complex simulation that runs until one side's army is eliminated. This simulation should handle armies of any size and plot the outcome.

This program should model battle outcomes, track results, and create visualizations of the simulated rounds.

In [149]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [150]:
#Ref.:https://www.w3schools.com/python/numpy/numpy_random.asp
# https://numpy.org/doc/2.0/reference/random/generated/numpy.random.seed.html
 

# I need to create two generators, one for attacker and another for defender. 
#np.random.randint can generate 2D arrays
#1 and 7 is min and max; size is an array size.

#attacker's results

n = 1000 # quantity of raunds
m = 2 #quantity of dices
np.random.seed(4)
dice_roll_attacker= np.random.randint(1,7, size=(n,m+1))
print(f'Attacker’s turns: first five rows:\n{dice_roll_attacker[0:5,:]}')

#defender's results 
np.random.seed(4)
dice_roll_defender= np.random.randint(1,7, size=(n,m))
print(f'Defender’s turns: first five rows:\n{dice_roll_defender[0:5,:]}')


Attacker’s turns: first five rows:
[[3 6 2]
 [1 1 3]
 [2 3 5]
 [6 2 1]
 [5 3 5]]
Defender’s turns: first five rows:
[[3 6]
 [2 1]
 [1 3]
 [2 3]
 [5 6]]


In [151]:
# Ref.:https://www.w3schools.com/python/numpy/numpy_array_sort.asp

# Attacker has 3 dice and I need to take for comparison 2 biggest numbers.
# I will sort the array.
# axis=1 for sorting by raws
# [:, ::-1] - to invert the order
# expression ::-1 refers to the elements within each row.
# This means that we want to select all elements, but in reverse order.

dice_roll_attacker = np.sort(dice_roll_attacker, axis=1 )[:, ::-1]
print(f'Sorted attacker\'s turns. First five rows:\n{dice_roll_attacker[0:5,:]}')

Sorted attacker's turns. First five rows:
[[6 3 2]
 [3 1 1]
 [5 3 2]
 [6 2 1]
 [5 5 3]]


In [152]:
#Now, when it sorted, I can take the rows with bigger numbers.
two_largest_attack= dice_roll_attacker[:,0:2]
print(f'Sorted and filtered attacker\'s turns. first five rows:\n{two_largest_attack[0:5,:]}')

Sorted and filtered attacker's turns. first five rows:
[[6 3]
 [3 1]
 [5 3]
 [6 2]
 [5 5]]


In [153]:
#Sorting defender's turns.
dice_roll_defender = np.sort(dice_roll_defender, axis=1 )[:,::-1]
print(f'Sorted defender\'s turns. First five rows:\n{dice_roll_defender[0:5,:]}')

Sorted defender's turns. First five rows:
[[6 3]
 [2 1]
 [3 1]
 [3 2]
 [6 5]]


In [154]:
#Now I will compare 2 arrays and create an array with the results.
#Creating an array with zeros.
#Ref.: https://stackoverflow.com/questions/568962/how-do-i-create-an-empty-array-and-then-append-to-it-in-numpy

attakers_wins = np.zeros(shape=(n,m),dtype=int)
#attakers_wins

defender_wins = np.zeros(shape=(n,m),dtype=int)
#defender_wins

In [155]:
# n = quantity of raunds 
# i - rows
# j - columns
for i in range (n):
    for j in range(2):
        if dice_roll_defender[i][j] >= dice_roll_attacker[i][j]:
            defender_wins[i][j]=1
        else:
             attakers_wins[i][j]=1
print(defender_wins[0:5,:])
print("___________")
print(attakers_wins[0:5,:])
    
    
    

[[1 1]
 [0 1]
 [0 0]
 [0 1]
 [1 1]]
___________
[[0 0]
 [1 0]
 [1 1]
 [1 0]
 [0 0]]
